In [1]:
import math
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import collections as col
import re
import random
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
#from sklearn.svm import LinearSVC
from sklearn import linear_model
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier

In [2]:
def load_data(train='Yes', test='No', validation='No'):
	"""Loads and returns datasets as required
	   Return empty lst for if 'No'
	"""
	if train=='Yes':
		df_train = pd.read_csv('dataset/train.csv', sep=',')
	else:
		df_train = []

	if test=='Yes':
		df_test = pd.read_csv('dataset/test.csv', sep=',')
	else:
		df_test = []

	if validation=='Yes':
		df_validation = pd.read_csv('dataset/validation.csv', sep=',')
	else:
		df_validation = []
	
	print('Data loaded', len(df_train), len(df_test), len(df_validation))
	return df_train, df_test, df_validation

In [3]:
df_train, df_test, df_validation= load_data('Yes', 'No', 'Yes')

Data loaded 2697738 0 299749


## pCTR, Logistic regression and Naive Bayes

In [ ]:
# Feature engineering
def label_encoder(df_column_nint, column_le= None): 
    if column_le== None:
        column_le = LabelEncoder()
        column_le.fit(df_column_nint.unique())
    if column_le!= None:
        pass
    df_column_int = column_le.transform(df_column_nint)
    return pd.DataFrame(df_column_int), column_le

def onehot_encoder(df_column_nohe, column_ohe= None): 
    if column_ohe== None:
        column_ohe = OneHotEncoder()
        column_ohe.fit(np.unique(df_column_nohe))
    if column_ohe!= None:
        pass
    df_column_ohe = column_ohe.transform(df_column_nohe)
    return pd.DataFrame(df_column_ohe.toarray()), column_ohe

In [4]:
df_LE_train = pd.DataFrame()
df_LE_train = df_train

# Y - Target
array_y = df_LE_train[['click']].as_matrix()
array_y_r = np.reshape(array_y, (-1))

# X - Features
lst_int_features = ['weekday', 'hour', 'slotwidth', 'slotheight', 'advertiser', 'city'] 
lst_nint_features = ['adexchange', 'slotformat', 'slotvisibility', 'useragent']
lst_nint_le_features = ['le_adexchange', 'le_slotformat', 'le_slotvisibility', 'le_useragent']

# Label encoding - NB
le_adexchange, le_a1 = label_encoder(df_LE_train['adexchange'])
le_slotformat, le_s2 = label_encoder(df_LE_train['slotformat'])
le_slotvisibility, le_s3 = label_encoder(df_LE_train['slotvisibility'])
le_useragent, le_u4 = label_encoder(df_LE_train['useragent'])

df_LE_train['le_adexchange']= le_adexchange
df_LE_train['le_slotformat']= le_slotformat
df_LE_train['le_slotvisibility']= le_slotvisibility
df_LE_train['le_useragent']= le_useragent

NameError: name 'label_encoder' is not defined

In [ ]:
lst_x_1 = ['advertiser']
lst_x_2 = lst_int_features
lst_x_3 = [lst_int_features[0], lst_int_features[1], lst_nint_le_features[0]]
lst_x_all = lst_int_features+ lst_nint_le_features

LE_array_x_1 = df_LE_train[lst_x_1].as_matrix()
LE_array_x_2 = df_LE_train[lst_x_2].as_matrix()
LE_array_x_3 = df_LE_train[lst_x_3].as_matrix()
LE_array_x_all = df_LE_train[lst_x_all].as_matrix()

In [ ]:
# One-hot encoding - LR
df_OHE_train = pd.DataFrame()
df_OHE_train = df_train

ohe_adexchange, _= onehot_encoder(df_LE_train['le_adexchange'].values)
ohe_slotformat, _= onehot_encoder(df_LE_train['le_slotformat'].values)
ohe_slotvisibility, _= onehot_encoder(df_LE_train['le_slotvisibility'].values)
ohe_useragent, _= onehot_encoder(df_LE_train['le_useragent'].values)

ohe_weekday, _= onehot_encoder(df_OHE_train['weekday'].values)
ohe_hour, _= onehot_encoder(df_OHE_train['hour'].values)
ohe_slotwidth, _= onehot_encoder(df_OHE_train['slotwidth'].values)
ohe_slotheight, _= onehot_encoder(df_OHE_train['slotheight'].values)
ohe_advertiser, _= onehot_encoder(df_OHE_train['advertiser'].values)
ohe_city, _= onehot_encoder(df_OHE_train['city'].values)

In [ ]:
df_OHE_train= pd.concat([df_OHE_train, ohe_adexchange], axis=1, join='inner')
df_OHE_train= pd.concat([df_OHE_train, ohe_slotformat], axis=1, join='inner')
df_OHE_train= pd.concat([df_OHE_train, ohe_slotvisibility], axis=1, join='inner')
df_OHE_train= pd.concat([df_OHE_train, ohe_useragent], axis=1, join='inner')

df_OHE_train= pd.concat([df_OHE_train, ohe_weekday], axis=1, join='inner')
df_OHE_train= pd.concat([df_OHE_train, ohe_hour], axis=1, join='inner')
df_OHE_train= pd.concat([df_OHE_train, ohe_slotwidth], axis=1, join='inner')
df_OHE_train= pd.concat([df_OHE_train, ohe_slotheight], axis=1, join='inner')
df_OHE_train= pd.concat([df_OHE_train, ohe_advertiser], axis=1, join='inner')
df_OHE_train= pd.concat([df_OHE_train, ohe_city], axis=1, join='inner')

In [ ]:
#OHE_array_x_1 = df_OHE_train[lst_x_1].as_matrix()
#OHE_array_x_2 = df_OHE_train[lst_x_2].as_matrix()
#OHE_array_x_3 = df_OHE_train[lst_x_3].as_matrix()
OHE_array_x_all = df_OHE_train.iloc[35:539].as_matrix()

In [ ]:
# Build model
def Naive_Bayes(array_x, array_y):
    NB_model = GaussianNB()
    NB_model.fit(array_x, array_y)
    return NB_model

def Logistic_Regression(array_x, array_y):
    LR_model = linear_model.LogisticRegression(penalty='l2')
    LR_model.fit(array_x, array_y)
    return LR_model

def AdaBoost_Classifier(array_x, array_y):
    AB_model = ensemble.AdaBoostClassifier()
    AB_model.fit(array_x, array_y)
    return AB_model

# Build with on 'advertiser' from validation set
#NB_model_1 = Naive_Bayes(array_x_1, array_y_r)
#NB_model_2 = Naive_Bayes(array_x_2, array_y_r)
NB_model_3 = Naive_Bayes(array_x_3, array_y_r)
NB_model_all = Naive_Bayes(array_x_all, array_y_r)

#LR_model_1 = Logistic_Regression(array_x_1, array_y_r)
#LR_model_2 = Logistic_Regression(array_x_2, array_y_r)
#LR_model_3 = Logistic_Regression(array_x_3, array_y_r)
LR_model_all = Logistic_Regression(array_x_all, array_y_r)

AB_model_all = AdaBoost_Classifier(array_x_all, array_y_r)

In [ ]:
le_adexchange, _ = label_encoder(df_validation['adexchange'], le_a1)
le_slotformat, _ = label_encoder(df_validation['slotformat'], le_s2)
le_slotvisibility, _ = label_encoder(df_validation['slotvisibility'], le_s3)
le_useragent, _ = label_encoder(df_validation['useragent'], le_u4)

df_validation['le_adexchange']= le_adexchange
df_validation['le_slotformat']= le_slotformat
df_validation['le_slotvisibility']= le_slotvisibility
df_validation['le_useragent']= le_useragent

In [ ]:
# Infer model
#NB_model_1pCTR_v = NB_model_1.predict_proba(df_validation[lst_x_1].as_matrix())[:, 1]
#NB_model_2pCTR_v = NB_model_2.predict_proba(df_validation[lst_x_2].as_matrix())[:, 1]
#NB_model_3pCTR_v = NB_model_3.predict_proba(df_validation[lst_x_3].as_matrix())[:, 1]
NB_model_allpCTR_v = NB_model_all.predict_proba(df_validation[lst_x_all].as_matrix())[:, 1]

#LR_model_1pCTR_v = LR_model_1.predict_proba(df_validation[lst_x_1].as_matrix())[:, 1]
#LR_model_2pCTR_v = LR_model_2.predict_proba(df_validation[lst_x_2].as_matrix())[:, 1]
#LR_model_3pCTR_v = LR_model_3.predict_proba(df_validation[lst_x_3].as_matrix())[:, 1]
LR_model_allpCTR_v = LR_model_all.predict_proba(df_validation[lst_x_all].as_matrix())[:, 1]

In [ ]:
# Evaluate model - CE
def evaluate_model1(df_val, array_pCTR):
    return -1/len(df_val) *sum((1- df_val['click'])* np.log(np.clip(array_pCTR, 1e-15 , 1)) +\
    (df_val['click'])* np.log(np.clip((1- array_pCTR), 1e-15 , 1)))

def evaluate_model2(df_val, array_pCTR):
    return -1/len(df_val) *sum((1- df_val['click'])* np.log(np.clip(array_pCTR, 1e-15 , 1)))

def evaluate_model3(df_val, array_pCTR):
    return -1/len(df_val) *sum(df_val['click']* np.log(np.clip((1- array_pCTR), 1e-15 , 1)))

print('NB')
#print(evaluate_model1(df_validation, NB_model_1pCTR_v))
#print(evaluate_model1(df_validation, NB_model_2pCTR_v))
#print(evaluate_model1(df_validation, NB_model_3pCTR_v))
print(evaluate_model1(df_validation, NB_model_allpCTR_v))
print('LR')
#print(evaluate_model1(df_validation, LR_model_1pCTR_v))
#print(evaluate_model1(df_validation, LR_model_2pCTR_v))
#print(evaluate_model1(df_validation, LR_model_3pCTR_v))
print(evaluate_model1(df_validation, LR_model_allpCTR_v))

In [ ]:
#.score
array_y = df_validation[['click']].as_matrix()
array_y_r = np.reshape(array_y, (-1))

print('NB')
#print(NB_model_1.score(df_validation[lst_x_1].as_matrix(), array_y_r))
#print(NB_model_2.score(df_validation[lst_x_2].as_matrix(), array_y_r))
#print(NB_model_3.score(df_validation[lst_x_3].as_matrix(), array_y_r))
print(NB_model_all.score(df_validation[lst_x_all].as_matrix(), array_y_r))
print('LR')
#print(LR_model_1.score(df_validation[lst_x_1].as_matrix(), array_y_r))
#print(LR_model_2.score(df_validation[lst_x_2].as_matrix(), array_y_r))
#print(LR_model_3.score(df_validation[lst_x_3].as_matrix(), array_y_r))
print(LR_model_all.score(df_validation[lst_x_all].as_matrix(), array_y_r))

In [ ]:
df_pCTRs= pd.DataFrame()
df_pCTRs= df_validation
#df_pCTRs['NB_model_1pCTR_v']= NB_model_1pCTR_v
#df_pCTRs['NB_model_2pCTR_v']= NB_model_2pCTR_v
#df_pCTRs['NB_model_3pCTR_v']= NB_model_3pCTR_v
df_pCTRs['NB_model_allpCTR_v']= NB_model_allpCTR_v
#df_pCTRs['LR_model_1pCTR_v']= LR_model_1pCTR_v
#df_pCTRs['LR_model_2pCTR_v']= LR_model_2pCTR_v
#df_pCTRs['LR_model_3pCTR_v']= LR_model_3pCTR_v
df_pCTRs['LR_model_allpCTR_v']= LR_model_allpCTR_v

In [ ]:
#df_temp1= df_pCTRs[['bidid', 'click', 'NB_model_1pCTR_v']].sort_values('NB_model_1pCTR_v', ascending=False)
#df_temp2= df_pCTRs[['bidid', 'click', 'NB_model_2pCTR_v']].sort_values('NB_model_2pCTR_v', ascending=False)
#df_temp3= df_pCTRs[['bidid', 'click', 'NB_model_3pCTR_v']].sort_values('NB_model_3pCTR_v', ascending=False)
df_temp4= df_pCTRs[['bidid', 'click', 'NB_model_allpCTR_v']].sort_values('NB_model_allpCTR_v', ascending=False)
#df_temp5= df_pCTRs[['bidid', 'click', 'LR_model_1pCTR_v']].sort_values('LR_model_1pCTR_v', ascending=False)
#df_temp6= df_pCTRs[['bidid', 'click', 'LR_model_2pCTR_v']].sort_values('LR_model_2pCTR_v', ascending=False)
#df_temp7= df_pCTRs[['bidid', 'click', 'LR_model_3pCTR_v']].sort_values('LR_model_3pCTR_v', ascending=False)
df_temp8= df_pCTRs[['bidid', 'click', 'LR_model_allpCTR_v']].sort_values('LR_model_allpCTR_v', ascending=False)

def count_clicks_order(df_pCTRs):
    counter= 0
    dic= {}
    for i in range(len(df_pCTRs)):
        if df_pCTRs[i:(i+1)]['click'].values== 1:
            counter+= 1
            dic[counter]= i
            if counter== 226:
                break
    return dic, counter

#dic1= count_clicks_order(df_temp1)
#dic2= count_clicks_order(df_temp2)
#dic3= count_clicks_order(df_temp3)
dic4= count_clicks_order(df_temp4)
#dic5= count_clicks_order(df_temp5)
#dic6= count_clicks_order(df_temp6)
#dic7= count_clicks_order(df_temp7)
dic8= count_clicks_order(df_temp8)

In [ ]:
x = []
y = []
#for i, j in dic1[0].items():
#    x.append(i)
#    y.append(j)
x1 = []
y1 = []
#for i, j in dic2[0].items():
#    x1.append(i)
#    y1.append(j)
x2 = []
y2 = []
#for i, j in dic3[0].items():
#    x2.append(i)
#    y2.append(j) 
x3 = []
y3 = []
for i, j in dic4[0].items():
    x3.append(i)
    y3.append(j) 
x4 = []
y4 = []
#for i, j in dic5[0].items():
#    x4.append(i)
#    y4.append(j) 
x6 = []
y6 = []
#for i, j in dic7[0].items():
#    x6.append(i)
#    y6.append(j) 
x7 = []
y7 = []
for i, j in dic8[0].items():
    x7.append(i)
    y7.append(j) 

In [ ]:
#plt.plot(y, x, label="NB_model_1pCTR_v")
#plt.plot(y2, x2, label="NB_model_3pCTR_v") 
plt.plot(y3, x3, label="NB_model_allpCTR_v")

#plt.plot(y4, x4, label="LR_model_1pCTR_v")
#plt.plot(y6, x6, label="LR_model_3pCTR_v")
plt.plot(y7, x7, label="LR_model_allpCTR_v")

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel('Validation impressions ordered by pCTR/ model')
plt.ylabel('Validation clicked impressions')
plt.show()

In [ ]:
# AUC - 0.75 our current best, paper 0.85
def AUC_accuracy(XX_model, array_x, array_y):
    fpr, tpr, thresholds = metrics.roc_curve(array_y, XX_model.predict_proba(array_x_all)[:, 1])
    return metrics.auc(fpr, tpr)

print(AUC_accuracy(NB_model_all, array_x_all, array_y_r))
print(AUC_accuracy(NB_model_3, array_x_3, array_y_r))
print(AUC_accuracy(LR_model_all, array_x_all, array_y_r))

## Linear bidding strategy

In [ ]:
df_int2= pd.DataFrame()
df_int2= df_validation
df_int2['pCTR']= NB_model_allpCTR_v
df_int2['pCTR_norm'] = df_int2['pCTR'].values/ df_int2['pCTR'].mean()

In [ ]:
def linear_bid(baseline_bid, imp_pCTR_norm):
    lin_bid = baseline_bid* imp_pCTR_norm
    return lin_bid

def nlinear_bid(baseline_bid, imp_pCTR_norm):
    lin_bid = baseline_bid* (imp_pCTR_norm**2)
    return nlin_bid

def bid_run(df_data_pCTR, baseline_bid, bid_strat= linear_bid, test_run= False, save_run= False):
    
    if test_run== True:    
        df_data_pCTR['imp_bid']= linear_bid(baseline_bid, df_data_pCTR['pCTR_norm'].values)
        
        df_output = pd.DataFrame()
        df_output[['bidid', 'bidprice']]= df_data_pCTR[['bidid', 'imp_bid']]
        
        if save_run== True:
            output_directory= 'Submission/Val/'
            output_filename= 'Group_XX.csv'
            df_output.to_csv(output_directory + output_filename)
            print('Submission file saved: ', os.getcwd(), output_directory, output_filename)
        if save_run== False:
            print('Submission file not saved')
        
        return df_data_pCTR
    
    if test_run== False:
        df_data_pCTR['imp_bid']= linear_bid(baseline_bid, df_data_pCTR['pCTR_norm'].values)
        df_data_pCTR['imp_h_bid']= np.where(df_data_pCTR['imp_bid'] >= df_data_pCTR['bidprice'], 'Y', 'N')
        
        total_cost = 0
        budget = 25000000
        
        df_data_pCTR['imp_win']= 'TBC'
        
        for index, row in df_data_pCTR.iterrows():
            if row['imp_h_bid']== 'Y':
                if (total_cost+ row['bidprice']) < budget:
                    df_data_pCTR.set_value(index,'imp_win', 'Y')
                    total_cost+= row['bidprice']
                else:
                    df_data_pCTR.set_value(index,'imp_win', 'Insuf/b')
            else:
                df_data_pCTR.set_value(index,'imp_win', 'N')
        
        return df_data_pCTR

In [ ]:
df_data_pCTR_bid = pd.DataFrame()
df_data_pCTR_bid = bid_run(df_int2, 1000, test_run= False)

In [ ]:
budget= 25000000

def CTR_evaluation(df_data_pCTR_bid):
    clicks = df_data_pCTR_bid['click'][(df_data_pCTR_bid['click'] == 1)].count()
    total = df_data_pCTR_bid['click'][(df_data_pCTR_bid['click'] != 10)].count()
    total_CTR = clicks/ total
    print('Sample average', total_CTR)
    
    win = df_data_pCTR_bid['click'][(df_data_pCTR_bid['imp_h_bid'] == 'Y')].count()
    win_click = df_data_pCTR_bid['click'][(df_data_pCTR_bid['imp_h_bid'] == 'Y') & (df_data_pCTR_bid.click == 1)].count()
    won_CTR = win_click/ win
    print('Win bids', won_CTR)
    
    loss = df_data_pCTR_bid['click'][(df_data_pCTR_bid['imp_h_bid'] == 'N')].count()
    loss_click = df_data_pCTR_bid['click'][(df_data_pCTR_bid['imp_h_bid'] == 'N') & (df_data_pCTR_bid.click == 1)].count()
    loss_CTR = loss_click/ loss
    print('Not won bids', loss_CTR)
    print('Won: Average ratio', won_CTR/ total_CTR)
    
    print('KPIs---')
    print('Number of clicks bid (NCB)', win_click)
    print('Total spend', df_data_pCTR_bid['bidprice'][(df_data_pCTR_bid['imp_h_bid'] == 'Y')].sum())
    print('Total spend (% budget)', (df_data_pCTR_bid['bidprice'][(df_data_pCTR_bid['imp_h_bid'] == 'Y')].sum()/ budget))
    print('Average CPM (Cost Per Mille)', '???')
    print('Average CPC (Cost Per Click)', (df_data_pCTR_bid['bidprice'][(df_data_pCTR_bid['imp_h_bid'] == 'Y')].sum())/ win_click)
    
CTR_evaluation(df_data_pCTR_bid)

In [ ]:
# Find other evaluation metric
# Optimise for baseline_bid
# Optimise for other pCRT